# Server

A C++ server application that establishes a secure SSL/TLS connection, listens for client messages, and echoes them back. It leverages OpenSSL for encryption and handles multiple client connections sequentially.

## Table of Contents
- [Overview](#overview)
- [Architecture](#architecture)
- [Dependencies](#dependencies)
- [Installation](#installation)
- [Usage](#usage)
- [Code Structure](#code-structure)
- [Security Considerations](#security-considerations)
- [License](#license)

## Overview

This server application performs the following operations:
- Initializes OpenSSL libraries.
- Creates and configures an SSL context with server certificates.
- Sets up a TCP socket to listen for incoming client connections.
- Accepts client connections and performs SSL handshakes.
- Reads messages from clients and echoes them back securely.
- Logs activities and handles errors appropriately.

## Architecture

```plaintext
+------------------+
|    Initialize    |
|      SSL         |
+--------+---------+
         |
         v
+--------+---------+
| Create SSL Context|
+--------+---------+
         |
         v
+--------+---------+
|  Create Socket   |
|    and Bind      |
+--------+---------+
         |
         v
+--------+---------+
|   Listen on Socket|
+--------+---------+
         |
         v
+--------+---------+
|   Accept Client   |
|   Connection      |
+--------+---------+
         |
         v
+--------+---------+       +-------------------------+
| SSL Handshake     |<----->|  Handle Client Messages |
+--------+---------+       +-------------------------+
         |
         v
+--------+---------+
|  Echo Message    |
+--------+---------+
         |
         v
+--------+---------+
|  Clean Up and    |
|  Shutdown        |
+------------------+
```

## Dependencies

- **Operating System**: Unix-like (Linux, macOS)
- **Compiler**: GCC or Clang with C++11 support
- **Libraries**:
  - OpenSSL
- **Certificates**:
  - `server.crt`: Server certificate in PEM format.
  - `server.key`: Server private key in PEM format.

## Installation

### Install OpenSSL

Ensure OpenSSL is installed on your system.

```bash
sudo apt-get update
sudo apt-get install libssl-dev
```

### Generate SSL Certificates

If you don't have SSL certificates, you can generate self-signed certificates for testing purposes.

```bash
openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout server.key -out server.crt
```

### Build the Application

```bash
./CREATE --build-project
```

## Usage

```bash
./server
```

The server will:
1. Initialize OpenSSL libraries.
2. Create and configure an SSL context with `server.crt` and `server.key`.
3. Bind to IP `192.168.2.2` on port `8085`.
4. Listen for incoming client connections.
5. Accept a client connection and perform an SSL handshake.
6. Read messages from the client and echo them back.
7. Handle multiple clients sequentially.
8. Log all activities and errors to the console.

## Code Structure

### `server.cpp`

```cpp
#include <iostream>
#include <cstring>
#include <sys/socket.h>
#include <netinet/in.h>
#include <unistd.h>
#include <openssl/ssl.h>
#include <openssl/err.h>
#include <arpa/inet.h> // for inet_addr

// Initialize OpenSSL libraries
void initialize_ssl()
{
    SSL_load_error_strings();
    OpenSSL_add_ssl_algorithms();
}

// Create SSL context using TLS server method
SSL_CTX *create_context()
{
    const SSL_METHOD *method = TLS_server_method();
    SSL_CTX *ctx = SSL_CTX_new(method);
    if (!ctx)
    {
        perror("Unable to create SSL context");
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }
    return ctx;
}

// Clean up OpenSSL resources
void cleanup_openssl()
{
    EVP_cleanup();
    ERR_free_strings();
}

// Configure SSL context with certificate and private key
void configure_context(SSL_CTX *ctx)
{
    SSL_CTX_set_ecdh_auto(ctx, 1);

    // Set the server certificate
    if (SSL_CTX_use_certificate_file(ctx, "server.crt", SSL_FILETYPE_PEM) <= 0)
    {
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }

    // Set the server private key
    if (SSL_CTX_use_PrivateKey_file(ctx, "server.key", SSL_FILETYPE_PEM) <= 0)
    {
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }
}

int main()
{
    // Initialize SSL libraries
    initialize_ssl();

    // Create and configure SSL context
    SSL_CTX *ctx = create_context();
    configure_context(ctx);

    // Create a TCP socket
    int sockfd = socket(AF_INET, SOCK_STREAM, 0);
    if (sockfd < 0)
    {
        perror("Unable to create socket");
        exit(EXIT_FAILURE);
    }

    // Define server address
    sockaddr_in addr;
    memset(&addr, 0, sizeof(addr));
    addr.sin_family = AF_INET;
    addr.sin_port = htons(8085);
    addr.sin_addr.s_addr = inet_addr("192.168.2.2");

    // Bind the socket to the specified IP and port
    if (bind(sockfd, (struct sockaddr *)&addr, sizeof(addr)) < 0)
    {
        perror("Bind failed");
        close(sockfd);
        exit(EXIT_FAILURE);
    }

    // Listen for incoming connections
    if (listen(sockfd, 1) < 0)
    {
        perror("Listen failed");
        close(sockfd);
        exit(EXIT_FAILURE);
    }

    std::cout << "Server is listening on 192.168.2.2:8085" << std::endl;

    while (1)
    {
        struct sockaddr_in client_addr;
        socklen_t len = sizeof(client_addr);

        // Accept a client connection
        int client = accept(sockfd, (struct sockaddr *)&client_addr, &len);
        if (client < 0)
        {
            perror("Unable to accept");
            continue;
        }

        // Create a new SSL structure for the connection
        SSL *ssl = SSL_new(ctx);
        SSL_set_fd(ssl, client);

        // Perform SSL handshake with the client
        if (SSL_accept(ssl) <= 0)
        {
            ERR_print_errors_fp(stderr);
        }
        else
        {
            char buffer[1024] = {0};

            // Read data from the client
            int bytes = SSL_read(ssl, buffer, sizeof(buffer));
            if (bytes > 0)
            {
                std::cout << "Client says: " << buffer << std::endl;

                // Echo the received message back to the client
                SSL_write(ssl, buffer, strlen(buffer));
            }
            else
            {
                ERR_print_errors_fp(stderr);
            }
        }

        // Clean up SSL and close the client socket
        SSL_free(ssl);
        close(client);
    }

    // Close the listening socket and free SSL context
    close(sockfd);
    SSL_CTX_free(ctx);
    cleanup_openssl();
    return 0;
}
```

### Function Descriptions

#### `initialize_ssl()`

**Purpose**: Initializes the OpenSSL library by loading error strings and adding all SSL algorithms. This is a prerequisite for any SSL operations.

**Implementation**:
```cpp
void initialize_ssl()
{
    SSL_load_error_strings();
    OpenSSL_add_ssl_algorithms();
}
```

#### `create_context()`

**Purpose**: Creates a new SSL context using the TLS server method. The context defines the SSL/TLS protocol versions and cipher suites that will be used.

**Implementation**:
```cpp
SSL_CTX *create_context()
{
    const SSL_METHOD *method = TLS_server_method();
    SSL_CTX *ctx = SSL_CTX_new(method);
    if (!ctx)
    {
        perror("Unable to create SSL context");
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }
    return ctx;
}
```

#### `configure_context(SSL_CTX *ctx)`

**Purpose**: Configures the SSL context with the server's certificate and private key. It also enables automatic selection of the appropriate elliptic curve for ECDH.

**Implementation**:
```cpp
void configure_context(SSL_CTX *ctx)
{
    SSL_CTX_set_ecdh_auto(ctx, 1);

    // Set the server certificate
    if (SSL_CTX_use_certificate_file(ctx, "server.crt", SSL_FILETYPE_PEM) <= 0)
    {
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }

    // Set the server private key
    if (SSL_CTX_use_PrivateKey_file(ctx, "server.key", SSL_FILETYPE_PEM) <= 0)
    {
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }
}
```

#### `cleanup_openssl()`

**Purpose**: Cleans up the OpenSSL library by freeing allocated resources and error strings.

**Implementation**:
```cpp
void cleanup_openssl()
{
    EVP_cleanup();
    ERR_free_strings();
}
```

#### `main()`

**Purpose**: The entry point of the server application. It initializes SSL, sets up the SSL context, creates and binds a TCP socket, listens for incoming connections, and handles client communications securely.

**Implementation**:
```cpp
int main()
{
    // Initialize SSL libraries
    initialize_ssl();

    // Create and configure SSL context
    SSL_CTX *ctx = create_context();
    configure_context(ctx);

    // Create a TCP socket
    int sockfd = socket(AF_INET, SOCK_STREAM, 0);
    if (sockfd < 0)
    {
        perror("Unable to create socket");
        exit(EXIT_FAILURE);
    }

    // Define server address
    sockaddr_in addr;
    memset(&addr, 0, sizeof(addr));
    addr.sin_family = AF_INET;
    addr.sin_port = htons(8085);
    addr.sin_addr.s_addr = inet_addr("192.168.2.2");

    // Bind the socket to the specified IP and port
    if (bind(sockfd, (struct sockaddr *)&addr, sizeof(addr)) < 0)
    {
        perror("Bind failed");
        close(sockfd);
        exit(EXIT_FAILURE);
    }

    // Listen for incoming connections
    if (listen(sockfd, 1) < 0)
    {
        perror("Listen failed");
        close(sockfd);
        exit(EXIT_FAILURE);
    }

    std::cout << "Server is listening on 192.168.2.2:8085" << std::endl;

    while (1)
    {
        struct sockaddr_in client_addr;
        socklen_t len = sizeof(client_addr);

        // Accept a client connection
        int client = accept(sockfd, (struct sockaddr *)&client_addr, &len);
        if (client < 0)
        {
            perror("Unable to accept");
            continue;
        }

        // Create a new SSL structure for the connection
        SSL *ssl = SSL_new(ctx);
        SSL_set_fd(ssl, client);

        // Perform SSL handshake with the client
        if (SSL_accept(ssl) <= 0)
        {
            ERR_print_errors_fp(stderr);
        }
        else
        {
            char buffer[1024] = {0};

            // Read data from the client
            int bytes = SSL_read(ssl, buffer, sizeof(buffer));
            if (bytes > 0)
            {
                std::cout << "Client says: " << buffer << std::endl;

                // Echo the received message back to the client
                SSL_write(ssl, buffer, strlen(buffer));
            }
            else
            {
                ERR_print_errors_fp(stderr);
            }
        }

        // Clean up SSL and close the client socket
        SSL_free(ssl);
        close(client);
    }

    // Close the listening socket and free SSL context
    close(sockfd);
    SSL_CTX_free(ctx);
    cleanup_openssl();
    return 0;
}
```

## Security Considerations

- **Certificate Validation**: Ensure that `server.crt` and `server.key` are securely stored within some enviornment variable.
- **Network Security**: Restrict access to the server port (`8085`) using firewalls or security groups to allow only trusted clients.